In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import time
import random
import os

# Part I: Stats Database ('22-'23)

In [ ]:
# Gets all the box score links from an inputted month's link
def get_box_score_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table with the schedule
    schedule_table = soup.find('table', {'id': 'schedule'})

    # Iterate through the rows
    box_score_links = []
    for row in schedule_table.find_all('tr'):
        # Extract the box score link
        box_score_cell = row.find('td', {'data-stat': 'box_score_text'})
        if box_score_cell:
            box_score_link = box_score_cell.find('a')['href']
            box_score_links.append('https://www.basketball-reference.com' + box_score_link)

    return box_score_links


In [ ]:
# Testing on October
october_links = get_box_score_links("https://www.basketball-reference.com/leagues/NBA_2023_games-october.html")
print(october_links[1:5])


['https://www.basketball-reference.com/boxscores/202210180GSW.html', 'https://www.basketball-reference.com/boxscores/202210190DET.html', 'https://www.basketball-reference.com/boxscores/202210190IND.html', 'https://www.basketball-reference.com/boxscores/202210190ATL.html']


In [ ]:
# Iterates over all months, retrieving all their box scores (up till Playoffs)
def get_all_box_score_links():
    # List to hold all the links
    all_links = []

    # Months we need to iterate over
    months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']

    # Base URL structure
    base_url = "https://www.basketball-reference.com/leagues/NBA_2023_games-{}.html"

    # Iterate over each month
    for month in months:
        month_url = base_url.format(month)
        links_for_month = get_box_score_links(month_url)

        # If it's April, filter out the games after April 9
        if month == 'april':
          links_for_month = [link for link in links_for_month if datetime.datetime.strptime(link.split('/')[-1][0:8], '%Y%m%d').day <= 9]


        # Extend the main list with the links for this month
        all_links.extend(links_for_month)

    return all_links

In [ ]:
# Testing to see if first, last, and length look right
all_links = get_all_box_score_links()
print(all_links[0])
print(all_links[len(all_links)-1])
print(len(all_links))

https://www.basketball-reference.com/boxscores/202210180BOS.html
https://www.basketball-reference.com/boxscores/202304090POR.html
1230


In [ ]:
def scrape_box_score_from_url(url):
    # Fetch the HTML content from the provided URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    soup = BeautifulSoup(response.text, 'html.parser')

    # Container for all the data
    data = []

    # Find all tables that have the basic game stats (for each team)
    for table in soup.find_all('table', {'class': 'stats_table'}):
        # Ensure we are in the right table
        caption = table.caption.text
        if "Basic and Advanced Stats" not in caption:
            continue

        # Extracting the full team name. We'll remove the last four words.
        words = caption.split()
        team_name = ' '.join(words[:-5])

        # Extract data for each player
        for row in table.tbody.find_all('tr'):
            player_data = {'team': team_name}

            player_cell = row.find('th', {'data-stat': 'player'})
            if player_cell and player_cell.a:
                player_data['player'] = player_cell.a.text
            else:
                continue  # Skip this row if no player name is found

            for cell in row.find_all('td'):
                data_stat = cell['data-stat']
                player_data[data_stat] = cell.text

            # Append the player's data to our main data container
            data.append(player_data)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Extract the date from the URL
    match_date = re.search(r"(\d{8})", url)
    if match_date:
        date_str = match_date.group(1)
        date_formatted = f"{date_str[4:6]}/{date_str[6:8]}/{date_str[2:4]}"
        df['date'] = date_formatted

    return df


In [ ]:
# Testing on sample game
scrape_box_score_from_url("https://www.basketball-reference.com/boxscores/202210180GSW.html")

,team,player,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,trb,ast,stl,blk,tov,pf,pts,plus_minus,reason,date
0,Los Angeles Lakers,Anthony Davis,35:36,10,22,.455,0,3,.000,7,...,6,0,4,1,3,2,27,-21,NaN,10/18/22
1,Los Angeles Lakers,LeBron James,35:12,12,26,.462,3,10,.300,4,...,15,8,0,0,5,2,31,-10,NaN,10/18/22
2,Los Angeles Lakers,Russell Westbrook,30:41,7,12,.583,1,3,.333,4,...,11,3,1,0,4,1,19,-6,NaN,10/18/22
3,Los Angeles Lakers,Lonnie Walker IV,29:17,2,7,.286,0,3,.000,1,...,3,5,1,1,2,0,5,-13,NaN,10/18/22
4,Los Angeles Lakers,Patrick Beverley,24:40,1,5,.200,1,5,.200,0,...,2,2,4,2,1,5,3,-5,NaN,10/18/22
5,Los Angeles Lakers,Kendrick Nunn,23:03,5,9,.556,3,6,.500,0,...,3,2,1,0,3,2,13,-19,NaN,10/18/22
6,Los Angeles Lakers,Matt Ryan,17:49,1,4,.250,1,3,.333,0,...,1,0,1,0,1,2,3,+1,NaN,10/18/22
7,Los Angeles Lakers,Austin Reaves,16:00,1,3,.333,0,2,.000,1,...,0,1,0,0,0,1,3,-6,NaN,10/18/22
8,Los Angeles Lakers,Juan Toscano-Anderson,14:18,0,3,.000,0,3,.000,0,...,4,0,0,0,2,1,0,0,NaN,10/18/22
9,Los Angeles Lakers,Wenyen Gabriel,7:48,0,1,.000,0,0,,0,...,1,1,0,0,0,2,0,0,NaN,10/18/22


In [ ]:
def gather_all_stats(all_links):
    # Initialize an empty list to store dataframes
    dfs = []

    # Iterate over all links
    for link in all_links:

        # Scrape the box score data and append to the list
        dfs.append(scrape_box_score_from_url(link))
        time.sleep(random.uniform(3,5))

    # Concatenate all the dataframes into one
    return pd.concat(dfs, ignore_index=True)


In [ ]:
# Testing by getting December stats (took like 20 min)
december_stats_df = gather_all_stats(december_links)


      team               player     mp   fg  fga fg_pct  fg3 fg3a fg3_pct  \
1   Dallas     Tim Hardaway Jr.  40:14    9   20   .450    6   14    .429   
2   Dallas  Dorian Finney-Smith  31:46    2    5   .400    2    4    .500   
3   Dallas    Spencer Dinwiddie  27:50    0    2   .000    0    1    .000   
4   Dallas        Dwight Powell  17:32    3    3  1.000    0    0           
5   Dallas       Christian Wood  34:35   10   13   .769    1    2    .500   
..     ...                  ...    ...  ...  ...    ...  ...  ...     ...   
95  Boston          Luke Kornet  10:27    0    0           0    0           
96  Boston           Sam Hauser   9:32    2    4   .500    1    3    .333   
97  Boston        Blake Griffin    NaN  NaN  NaN    NaN  NaN  NaN     NaN   
98  Boston       Justin Jackson    NaN  NaN  NaN    NaN  NaN  NaN     NaN   
99  Boston    Mfiondu Kabengele    NaN  NaN  NaN    NaN  NaN  NaN     NaN   

     ft  ...  trb  ast  stl  blk  tov   pf  pts plus_minus        reason  \

In [ ]:
# Running on all links (took like 1.5 hrs)
all_stats_df = gather_all_stats(all_links)

In [ ]:
print(all_stats_df)

NameError: ignored

In [ ]:
31513/1230

25.620325203252033

# Part II: Bdays

In [ ]:
# Function to get player links from the main page
def get_player_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table with the player data
    players_table = soup.find('table', {'id': 'per_game_stats'})

    # Extract player links
    player_links = {}
    for row in players_table.tbody.find_all('tr', class_='full_table'):
        player_name = row.find('td', {'data-stat': 'player'}).text
        player_link = 'https://www.basketball-reference.com' + row.find('td', {'data-stat': 'player'}).a['href']
        player_links[player_name] = player_link

    return player_links

In [ ]:
def get_player_birthday(url):
    """Given the URL to a player's bio page, return the player's birthdate."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract player birthday using the 'id' attribute
    birth_date_element = soup.find('span', {'id': 'necro-birth'})

    if birth_date_element:
        return birth_date_element.get('data-birth')
    return None

In [ ]:
# Test
sample_precious_achiuwa_bday = get_player_birthday("https://www.basketball-reference.com/players/a/achiupr01.html")
print(sample_precious_achiuwa_bday)

1999-09-19


In [ ]:
# Function to iterate over all the player links, scrape their bio pages, and retrieve their birthdays
def get_all_players_birthdays(url):
    player_links = get_player_links(url)
    players_birthdays = {}

    for player, link in player_links.items():
        players_birthdays[player] = get_player_birthday(link)

        # Add a delay to avoid rate limiting
        time.sleep(random.uniform(3, 5))

    # Convert the dictionary to a dataframe
    df = pd.DataFrame(list(players_birthdays.items()), columns=['Player', 'Birthday'])

    return df

In [ ]:
# Call the main function
all_players_url = "https://www.basketball-reference.com/leagues/NBA_2023_per_game.html"
bdays_df = get_all_players_birthdays(all_players_url)
print(bdays_df)

               Player    Birthday
0    Precious Achiuwa  1999-09-19
1        Steven Adams  1993-07-20
2         Bam Adebayo  1997-07-18
3        Ochai Agbaji  2000-04-20
4        Santi Aldama  2001-01-10
..                ...         ...
534    Thaddeus Young  1988-06-21
535        Trae Young  1998-09-19
536    Omer Yurtseven  1998-06-19
537       Cody Zeller  1992-10-05
538       Ivica Zubac  1997-03-18

[539 rows x 2 columns]


# Part III: Downloading as CSVs to R

In [ ]:
from google.colab import files
bdays_df.to_csv('player_bdays.csv', index=False)
files.download('player_bdays.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
all_stats_df.to_csv('player_stats.csv', index=False)
bdays_df.to_csv('player_bdays.csv', index=False)
os.getcwd()



'/content'

In [ ]:
all_stats_df.to_csv('player_stats.csv', index=False)
files.download('player_stats.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>